In [1]:
!pip install torchsummary
!pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 27.5 MB/s eta 0:00:00


In [2]:
start= 0
end = 3

In [3]:
import torch
import numpy as np
from torch.utils.data import (
    Dataset,
    DataLoader,
) 
import pickle


import os

import math
import torch.nn as nn
import torch.nn.functional as F

from torchvision.transforms import Compose, Lambda, Grayscale,Normalize, CenterCrop,Resize,ConvertImageDtype,ToTensor
from torchvision.io import read_video
from tqdm import tqdm

from collections import OrderedDict

import torch.optim as optim

from torch.autograd import Variable


import torchvision.models as models

import sys
from torchsummary import summary






class SignDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y


    def __len__(self):

        return len(self.y)

        # length = 0
        # with open(self.file, 'rb') as f:
        #     data = pickle.load(f)

        # for x in data:
        #     length += len(data[x])
        # return length


    def __getitem__(self, index):
        return self.x[index], self.y[index]
        

In [4]:
IMAGE_HEIGHT = 512
IMAGE_WIDTH = 512
IMAGE_CHANNEL = 1
NUM_FRAMES = 10
NUM_CLASSES = 61



train_inputs =[] #x
train_classes = [] #y

def preprocess(batch,mean,std):
    transforms = Compose(
        [
            
            Lambda(lambda x: x/255.0),
            CenterCrop(size=(720, 900)),
            
            Normalize(mean=mean, std=std),  # map [0, 1] into [-1, 1]
            Grayscale(num_output_channels=1),
            Resize(size=(512, 512)),
        ]
    )
    batch = transforms(batch)
    return batch



def lp_video(filename,start_time, end_time):
    frames, _, _ = read_video("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4",start_pts=start_time/1000.0,end_pts= end_time/1000.0,pts_unit='sec')
    
    frames = frames.permute(0, 3, 1, 2)  # (N, H, W, C) -> (N, C, H, W)
    frames = ConvertImageDtype(torch.float32)(frames)
    

    
  
            
    std, mean = torch.std_mean(frames)
    std = std/255.0
    mean = mean/255.0

    # tList = [m for m in torch.unbind(video_data, dim=4) ]
    # y = torch.stack(tList, dim=0)
        
            
            
    img1_batch = preprocess( frames, mean, std)
    
    return img1_batch

def load_dataloaders(batch_size,start,end):

    time_start = []
    time_end = []

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        
        print(" \n file:",key)
    
        for x in data[key]:
            img_cls = x[0]
            time_start.append(x[1])
            time_end.append(x[2])
            
            
            # start_time = x[1]
            # end_time = x[2]
            
            vid = lp_video(key,x[1], x[2])

            for m in vid:
                train_classes.append(img_cls)
                train_inputs.append(m)


            #some negative classes too
        for i in range(0,5):
            if  time_start[i+1]-time_end[i]>800:
                start_t = time_end[i]+500
                end_t = time_end[i]+550
                vid = lp_video(key,start, end)
                for m in vid:
                    print("here")
                    train_classes.append(60)
                    train_inputs.append(m)

            
        
            

    signds = SignDataset(train_inputs, train_classes)
    trainlen = int(len(signds)*0.8)
    torch.manual_seed(0)
    
    train_set, val_test_set = torch.utils.data.random_split(signds, [trainlen, len(signds)-trainlen])
    trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

    valloader = DataLoader(val_test_set, batch_size=batch_size, shuffle=True, num_workers=2)
  
    return trainloader, valloader
        
    

In [5]:
"""
In ResNet, we see how the skip connection added as identity function from the inputs
to interact with the Conv layers. But in DenseNet, we see instead of adding skip 
connection to Conv layers, we can append or concat the output of identity function
with output of Conv layers.

In ResNet, it is little tedious to make the dimensions to match for adding the skip
connection and Conv Layers, but it is much simpler in DenseNet, as we concat the 
both the X and Conv's output.

The key idea or the reason its called DenseNet is because the next layers not only get
the input from previous layer but also preceeding layers before the previous layer. So 
the next layer becomes dense as it loaded with output from previous layers.

Check Figure 7.7.2 from https://d2l.ai/chapter_convolutional-modern/densenet.html for 
why DenseNet is Dense?

Two blocks comprise DenseNet, one is DenseBlock for concat operation and other is 
transition layer for controlling channels meaning dimensions (recall 1x1 Conv).
"""
import math
import torch
import torch.nn as nn
from torch.nn import functional as F
"""
In ResNet, we see how the skip connection added as identity function from the inputs
to interact with the Conv layers. But in DenseNet, we see instead of adding skip 
connection to Conv layers, we can append or concat the output of identity function
with output of Conv layers.

In ResNet, it is little tedious to make the dimensions to match for adding the skip
connection and Conv Layers, but it is much simpler in DenseNet, as we concat the 
both the X and Conv's output.

The key idea or the reason its called DenseNet is because the next layers not only get
the input from previous layer but also preceeding layers before the previous layer. So 
the next layer becomes dense as it loaded with output from previous layers.

Check Figure 7.7.2 from https://d2l.ai/chapter_convolutional-modern/densenet.html for 
why DenseNet is Dense?

Two blocks comprise DenseNet, one is DenseBlock for concat operation and other is 
transition layer for controlling channels meaning dimensions (recall 1x1 Conv).
"""
import math
import torch
import torch.nn as nn
from torch.nn import functional as F

class Bottleneck(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(Bottleneck, self).__init__()
        interChannels = 4*growthRate
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, interChannels, kernel_size=1,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(interChannels)
        self.conv2 = nn.Conv2d(interChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat((x, out), 1)
        return out

class SingleLayer(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(SingleLayer, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = torch.cat((x, out), 1)
        return out

class Transition(nn.Module):
    def __init__(self, nChannels, nOutChannels):
        super(Transition, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, nOutChannels, kernel_size=1,
                               bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = F.avg_pool2d(out, 2)
        return out


class DenseNet(nn.Module):
    def __init__(self, input_channel=1, growthRate=12, depth=10, reduction=0.5, n_classes=61, bottleneck=True):
        super(DenseNet, self).__init__()

        nDenseBlocks = (depth-4) // 3
        if bottleneck:
            nDenseBlocks //= 2

        nChannels = 2*growthRate
        self.conv1 = nn.Conv2d(input_channel, nChannels, kernel_size=3, padding=1,
                               bias=False)
        self.dense1 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans1 = Transition(nChannels, nOutChannels)

        nChannels = nOutChannels
        self.dense2 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans2 = Transition(nChannels, nOutChannels)

        nChannels = nOutChannels
        self.dense3 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans3 = Transition(nChannels, nOutChannels)
        
        nChannels = nOutChannels
        self.dense4 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate

        self.bn1 = nn.BatchNorm2d(nChannels)
        self.fc = nn.Linear(nChannels, n_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()

    def _make_dense(self, nChannels, growthRate, nDenseBlocks, bottleneck):
        layers = []
        for i in range(int(nDenseBlocks)):
            if bottleneck:
                layers.append(Bottleneck(nChannels, growthRate))
            else:
                layers.append(SingleLayer(nChannels, growthRate))
            nChannels += growthRate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.trans3(self.dense3(out))
        out = self.dense4(out)
        out = F.relu(self.bn1(out))
        out = torch.squeeze(F.adaptive_avg_pool2d(out, 1))
        out = F.log_softmax(self.fc(out))

        return out

In [6]:
def evaluate(outputs: Variable, labels: Variable) -> float:
    """Evaluate neural network outputs against non-one-hotted labels."""
    Y = labels.numpy()
    Yhat = np.argmax(outputs, axis=1)
    return float(np.sum(Yhat == Y))

In [7]:

def test(model, device, loader, mode="Validate"):
    
    
    model.eval()

    correct = 0
    num_samples = 0
    
        
    for batch_idx, (data, target) in enumerate(loader):
        with torch.no_grad():    
            data = data.to(device)
            target = (target).to(device)

            output = model(data.half())  
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            #print("correct", correct)

    acc = 100.0 * correct / num_samples
    
    print('{} Accuracy: {}/{} ({:.0f}%)'.format(
                mode,correct, num_samples,
                100. * correct / num_samples, acc))

In [8]:
def trainonval(model, device,validloader, optimizer, criterion, epochs):

    
    model.train()

    
    
    print("Training on Validation Set starts with lr",optimizer.param_groups[0]['lr'])
    for epoch in range(epochs):
        correct = 0
        num_samples = 0
        train_loss = 0
        
        for batch_idx, (data, target) in enumerate(validloader):
            
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data.half())
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.item()
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            
            
            
        
        #print(scheduler.get_last_lr())
        train_loss /= num_samples
        
        if (100 * correct / num_samples) >= 90:
            
            print('Epoch: {} , Training Accuracy on Val set: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,100. * correct / num_samples, train_loss))
            test(model, device, testloader, mode = "Test after Training on validation set")
                
            return model, optimizer
            
    
        print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,
                100. * correct / num_samples, train_loss))
        
    return model, optimizer

In [9]:

    


def train(model, device, train_loader,validloader,scheduler, optimizer,criterion, epochs):
    print("Train start")
    breakout = False
    model.half()
    model.cuda()

    
  

    model.train()

    
    
    for epoch in range(epochs):
        correct = 0
        num_samples = 0
        train_loss = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data.half())
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.item()
            
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            
        #if optimizer.param_groups[0]['lr'] == 1e-2:    
        scheduler.step()
        #print("lr:",optimizer.param_groups[0]['lr'])
        
        
        train_loss /= num_samples
        
        if (100 * correct / num_samples) >= 95:
            
            print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,100. * correct / num_samples, train_loss))
            
            test(model,device,validloader, mode = "Validating")
#             test(model, device, testloader, mode = "Test before Training on validation set")
           # model, optimizer = trainonval(model, device, validloader, optimizer, criterion, epochs)
#             test(model, device, testloader, mode = "Test after training on validation set")
            
            
            
            torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer1.pt")
            
            breakout = True
            print("breakout")
            return
            
            
            
       
        

        print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,
                100. * correct / num_samples, train_loss))
        if (((epoch+1)%5) == 0) :
            test(model,device,validloader)
            model.train()
            
    if breakout:
            print("breakout")
            return
    
    
    
    #model, optimizer = trainonval(model, device, validloader, optimizer, criterion, epochs)
    test(model, device, validloader, mode= "test set ")
    
    
        
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer1.pt")
    
    
        


In [10]:
n_classes = 61


model = nn.Sequential(OrderedDict([
    ('frontend',DenseNet()),
]))





# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer and learning rate
optimizer = optim.SGD(
    [
        
        {"params": model.frontend.parameters(), "lr": 1e-1},
  ],
  momentum = 0.9
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
# state = torch.load("../input/sign-classification/model_optimizer1.pt")
# model.load_state_dict(state['model_state_dict'])
# model.half()
# model.cuda()
# optimizer.load_state_dict(state['optimizer_state_dict'])

In [11]:

trainloader, valloader = load_dataloaders(batch_size=32,start=start, end=end)#73 not included 


 
 file: p01_n002
here
here
here
 
 file: p06_n019
here
here
here
 
 file: p06_n007
here
here
here


In [12]:
#print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))

In [13]:
#summary(model, (1, 512, 512), device='cpu')

In [14]:
train(model,device,trainloader,valloader,scheduler,optimizer,criterion,50)

Train start


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:155: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1 , Training Accuracy: 91/755 (12%) Training Loss: 0.115403
Epoch: 2 , Training Accuracy: 150/755 (20%) Training Loss: 0.090576
Epoch: 3 , Training Accuracy: 146/755 (19%) Training Loss: 0.084564
Epoch: 4 , Training Accuracy: 156/755 (21%) Training Loss: 0.081271
Epoch: 5 , Training Accuracy: 148/755 (20%) Training Loss: 0.079320
Validate Accuracy: 32/189 (17%)
Epoch: 6 , Training Accuracy: 167/755 (22%) Training Loss: 0.076676
Epoch: 7 , Training Accuracy: 166/755 (22%) Training Loss: 0.073272
Epoch: 8 , Training Accuracy: 189/755 (25%) Training Loss: 0.071267
Epoch: 9 , Training Accuracy: 185/755 (25%) Training Loss: 0.070034
Epoch: 10 , Training Accuracy: 194/755 (26%) Training Loss: 0.073555
Validate Accuracy: 32/189 (17%)
Epoch: 11 , Training Accuracy: 184/755 (24%) Training Loss: 0.074395
Epoch: 12 , Training Accuracy: 213/755 (28%) Training Loss: 0.067698
Epoch: 13 , Training Accuracy: 193/755 (26%) Training Loss: 0.068218
Epoch: 14 , Training Accuracy: 170/755 (23%) Trai

In [15]:
#from 21 15 epochs